In [1]:
from functools import cache

import numpy as np
import networkx as nx
from tqdm import tqdm

In [2]:
def parse_input(file):
    with open(file) as file_in:
        codes = file_in.read().splitlines()
    return codes

In [3]:
def keypad_to_graph(keypad_str):
    grid = np.array([list(row) for row in keypad_str.splitlines()])
    rows, cols = grid.shape
    G = nx.Graph()
    coords_to_nodes = {}

    for r in range(rows):
        for c in range(cols):
            coord = (r, c)
            node = grid[r, c]

            if node == 'Z':  # Ignore specific nodes
                continue

            coords_to_nodes[coord] = node.item()

            # Add edges to neighbors
            for dr, dc in move2dir:
                nr, nc = r + dr, c + dc
                if 0 <= nr < rows and 0 <= nc < cols:
                    neighbor_coord = (nr, nc)
                    neighbor_node = grid[nr, nc]
                    if neighbor_node != 'Z':  # Ignore specific neighbors
                        G.add_edge(coord, neighbor_coord)

    nodes_to_coords = {v: k for k, v in coords_to_nodes.items()}

    return G, coords_to_nodes, nodes_to_coords

In [4]:
@cache
def compute_all_shortest_paths(G, source, target):
    return list(nx.all_shortest_paths(G, source, target))

In [5]:
def all_shortest_paths_to_directions(code, G, nodes2coords, move2dir):
    all_paths = [[(nodes2coords[code[0]], False)]]  # Start with the initial node
    directions_list = []  # Final list of directions

    for i in range(len(code) - 1):
        current_coords, next_coords = nodes2coords[code[i]], nodes2coords[code[i + 1]]

        # Skip if the coordinates are the same
        if current_coords == next_coords:
            for path in all_paths:
                path.append((current_coords, True))
            continue

        # Get all shortest paths between the current and next coordinates
        segment_paths = compute_all_shortest_paths(G, current_coords, next_coords)

        # Build all combinations of current paths with the new segment paths
        new_paths = []
        for existing_path in all_paths:
            for segment in segment_paths:
                # Append segment nodes and actions
                new_segment = [((x, y), (x, y) == segment[-1]) for (x, y) in segment[1:]]
                new_paths.append(existing_path + new_segment)
        all_paths = new_paths  # Update paths with the expanded ones

    # Convert paths to directions
    for sp in all_paths:
        directions = []
        for i in range(1, len(sp)):
            (x_current, y_current), action_current = sp[i]
            (x_prev, y_prev), __ = sp[i - 1]
            move = (x_current - x_prev, y_current - y_prev)
            if move != (0, 0):  # Add direction for movement
                directions.append(move2dir[move])
            if action_current:  # Add 'A' for actions
                directions.append('A')
        directions_list.append(''.join(directions))

    return directions_list

In [6]:
def one_shortest_path_to_directions(code, G, nodes2coords, move2dir):
    directions = []  # Final string of directions

    for i in range(len(code) - 1):
        current_coords, next_coords = nodes2coords[code[i]], nodes2coords[code[i + 1]]

        # Skip if the coordinates are the same
        if current_coords == next_coords:
            directions.append('A')  # Action at the same node
            continue

        # Compute the shortest path for the current pair of coordinates
        sp = nx.shortest_path(G, source=current_coords, target=next_coords)

        # Convert the path to directions
        for j in range(1, len(sp)):
            x_prev, y_prev = sp[j - 1]
            x_current, y_current = sp[j]
            move = (x_current - x_prev, y_current - y_prev)
            directions.append(move2dir[move])

        # Add action 'A' at the end of the segment
        directions.append('A')

    return ''.join(directions)

In [7]:
def main(file, n_intermediary_robots):
    codes = parse_input(file)

    total_complexity = 0
    for code in codes:
        code_up = 'A' + code
        dirs = all_shortest_paths_to_directions(code_up, G_num_keypad, num2coords, move2dir)

        for __ in range(n_intermediary_robots):
            dirs = ['A' + dir for dir in dirs]
            dirs_new = []
            min_len_dirs = min(len(d) for d in dirs)
            for dir in dirs:
                if len(dir) == min_len_dirs:
                    dirs_new.extend(all_shortest_paths_to_directions(dir, G_dir_keypad, dir2coords, move2dir))
                dirs = dirs_new

        dirs = ['A' + dir for dir in dirs]
        dirs_final = []
        min_len_dirs = min(len(d) for d in dirs)
        for dir in dirs:
            if len(dir) == min_len_dirs:
                dirs_final.append(one_shortest_path_to_directions(dir, G_dir_keypad, dir2coords, move2dir))

        len_shortest_seq = min(len(d) for d in dirs_final)
        code_numeric_part = int(''.join(char for char in list(code.lstrip("0")) if char.isnumeric()))
        total_complexity += len_shortest_seq * code_numeric_part

    return total_complexity

In [8]:
move2dir = {(-1, 0): '^', (1, 0): 'v', (0, -1): '<', (0, 1): '>'}
G_num_keypad, coords2num, num2coords = keypad_to_graph('789\n456\n123\nZ0A')
G_dir_keypad, coords2dir, dir2coords = keypad_to_graph('Z^A\n<v>')

precomputed_all_sp_num = dict(nx.all_pairs_shortest_path(G_num_keypad))
precomputed_all_sp_dir = dict(nx.all_pairs_shortest_path(G_dir_keypad))

In [9]:
assert main('example1.txt', n_intermediary_robots=1) == 126384

In [39]:
main('input.txt', n_intermediary_robots=1)

162740

In [ ]:
main('input.txt', n_intermediary_robots=25)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f2464474290>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
